# 01 – Tabular EDA & Feature Engineering

This notebook focuses on **Exploratory Data Analysis (EDA)** and **feature engineering** for tabular data using the `ml_tabular` template.

We will:

1. Load configuration and data
2. Inspect schema, types, and basic statistics
3. Analyze missing values and target distribution
4. Explore feature–target relationships
5. Explore correlations and potential leakage
6. Prototype basic feature engineering
7. (Optionally) write updated feature config back to YAML

The goal is not just to look at pretty plots, but to make **concrete decisions** about:

- Which columns to use
- Which feature types they are (numeric / categorical / datetime)
- What transformations we might want (log transforms, date decomposition, etc.)
- How to reflect those decisions in the project config and pipelines.

## 0. Setup

We assume you have installed the project (from the repo root):

```bash
pip install -e .[dev]
```

And that you have a tabular config at:

- `configs/tabular/train_tabular_baseline.yaml`

This notebook will *not* run long training jobs; it focuses on **understanding the data** and **prototyping feature ideas**.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ml_tabular import (
    get_config,
    get_paths,
    get_logger,
)

LOGGER = get_logger(__name__)
PROJECT_ROOT = Path.cwd()
CONFIG_PATH = PROJECT_ROOT / "configs" / "tabular" / "train_tabular_baseline.yaml"

assert CONFIG_PATH.exists(), f"Config not found: {CONFIG_PATH}"

cfg = get_config(config_path=CONFIG_PATH, env="dev", force_reload=True)
paths = get_paths(config_path=CONFIG_PATH, env="dev", force_reload=True)

cfg_dict = cfg.to_dict()
tab_cfg = cfg_dict.get("tabular", {})

tab_cfg

## 1. Load data

We use the dataset path specified in the tabular config, typically something like:

- `data/train.csv`

We also validate that the feature and target columns listed in config exist in the dataframe.

In [ ]:
dataset_csv = tab_cfg["dataset_csv"]
data_path = paths.data_dir / dataset_csv
assert data_path.exists(), f"Dataset not found: {data_path}"

df = pd.read_csv(data_path)
LOGGER.info("Loaded dataset with shape: %s", df.shape)
df.head()

Check that config columns exist and log them for reference.

In [ ]:
feature_columns = tab_cfg.get("feature_columns") or []
target_column = tab_cfg.get("target_column")

print("Configured feature columns (from YAML):", feature_columns)
print("Configured target column:", target_column)
print("DataFrame columns:", list(df.columns))

missing_features = [c for c in feature_columns if c not in df.columns]
missing_target = target_column not in df.columns if target_column is not None else True

print("Missing features:", missing_features)
print("Missing target?", missing_target)

if missing_features:
    LOGGER.warning("Some configured features are missing from the dataset: %s", missing_features)
if missing_target:
    LOGGER.warning("Configured target column '%s' is missing from the dataset.", target_column)

df.shape

## 2. High-level schema and types

We start with basic schema checks:

- Column names
- Dtypes
- Number of unique values (for categorical-ish columns)
- Basic `describe()` statistics

This helps us decide which columns should be treated as **numeric**, **categorical**, or **datetime**.

In [ ]:
df.info()

In [ ]:
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
non_numeric_cols = [c for c in df.columns if c not in numeric_cols]

print("Numeric columns:", numeric_cols)
print("Non-numeric columns:", non_numeric_cols)

df[numeric_cols].describe().T

In [ ]:
unique_counts = df[non_numeric_cols].nunique(dropna=True).sort_values()
unique_counts.to_frame("n_unique").T if unique_counts.empty else unique_counts.to_frame("n_unique")

> **Guideline:**
> - Low-cardinality non-numeric columns (e.g. `n_unique < 50`) often make good candidates for **categorical** features.
> - Columns that *look* like dates (e.g. strings like `2023-01-01`) should typically be parsed to `datetime64` and treated as **datetime**.

We can attempt to automatically parse obvious datetime columns next (purely for exploration; the proper handling can be wired into your config and pipelines).

In [ ]:
def try_parse_dates(df: pd.DataFrame, max_cols: int = 10) -> pd.DataFrame:
    """Attempt to parse object columns as datetimes (for EDA only).

    We only try the first `max_cols` object columns to avoid surprises on huge datasets.
    """
    df_copy = df.copy()
    obj_cols = df_copy.select_dtypes(include=["object", "string"]).columns.tolist()[:max_cols]
    for col in obj_cols:
        try:
            parsed = pd.to_datetime(df_copy[col], errors="raise")
        except Exception:
            continue
        else:
            LOGGER.info("Parsed column '%s' as datetime for EDA", col)
            df_copy[col] = parsed
    return df_copy

df_eda = try_parse_dates(df)
df_eda.dtypes.value_counts()

## 3. Missing values analysis

Understanding missingness is crucial both for **feature engineering** and **data quality**.

We’ll compute:
- Count and percentage of missing values per column
- Simple bar plot of missingness (if any).

In [ ]:
missing_counts = df_eda.isna().sum()
missing_pct = (missing_counts / len(df_eda)) * 100.0

missing_df = (
    pd.DataFrame({"missing_count": missing_counts, "missing_pct": missing_pct})
    .sort_values("missing_pct", ascending=False)
)

missing_df.head(20)

In [ ]:
high_missing = missing_df[missing_df["missing_pct"] > 0]
if not high_missing.empty:
    plt.figure(figsize=(8, max(3, len(high_missing) * 0.3)))
    plt.barh(high_missing.index, high_missing["missing_pct"])
    plt.xlabel("Missing %")
    plt.title("Missing values per column")
    plt.gca().invert_yaxis()
    plt.grid(axis="x", linestyle="--", alpha=0.5)
    plt.show()
else:
    print("No missing values detected.")

> **Questions to consider:**
> - Which columns have high missingness (e.g. > 30%) and might not be worth keeping?
> - For the remaining ones, do we prefer **imputation**, **special categories** (for categoricals), or domain-specific logic?

Your answers should inform both your **feature configuration** and how you design your **sklearn/PyTorch pipelines** downstream.

## 4. Target distribution and basic feature–target relationship

We analyze the target differently depending on the **task type**:

- **Regression**: histogram + summary stats
- **Binary classification**: class balance
- **Multiclass classification**: class counts

Then we’ll look at a few simple feature–target relationships (e.g., numeric features vs target).

In [ ]:
task_type = tab_cfg.get("task_type", "binary")
target_column = tab_cfg.get("target_column")

if target_column is None or target_column not in df_eda.columns:
    print("Target column not found; skipping target distribution analysis.")
else:
    y = df_eda[target_column]
    print("Target dtype:", y.dtype)
    print("Unique values (head):", y.unique()[:20])

    if task_type == "regression":
        plt.figure(figsize=(6, 4))
        plt.hist(y.dropna(), bins=30, edgecolor="black", alpha=0.7)
        plt.xlabel(target_column)
        plt.ylabel("Count")
        plt.title("Target distribution (regression)")
        plt.grid(axis="y", linestyle="--", alpha=0.5)
        plt.show()

        display(y.describe())

    else:
        value_counts = y.value_counts(dropna=False)
        display(value_counts.to_frame("count"))

        plt.figure(figsize=(6, 4))
        plt.bar(value_counts.index.astype(str), value_counts.values)
        plt.xlabel(target_column)
        plt.ylabel("Count")
        plt.title(f"Target distribution ({task_type})")
        plt.grid(axis="y", linestyle="--", alpha=0.5)
        plt.show()

### 4.1 Simple numeric feature vs target views

We can look at a few numeric features to get an intuition of how they relate to the target.

- For regression, scatter plots
- For classification, box plots per class (or simple groupby means)

In [ ]:
if target_column is None or target_column not in df_eda.columns:
    print("Skipping numeric feature vs target analysis; target not available.")
else:
    numeric_cols_no_target = [c for c in numeric_cols if c != target_column]
    cols_to_plot = numeric_cols_no_target[:4]

    if not cols_to_plot:
        print("No numeric features (other than target) found.")
    else:
        n_cols = len(cols_to_plot)
        plt.figure(figsize=(4 * n_cols, 4))
        for i, col in enumerate(cols_to_plot, start=1):
            plt.subplot(1, n_cols, i)
            if task_type == "regression":
                plt.scatter(df_eda[col], df_eda[target_column], alpha=0.5)
                plt.xlabel(col)
                plt.ylabel(target_column)
            else:
                # For classification, plot per-class means as a quick heuristic
                means = df_eda.groupby(target_column)[col].mean()
                plt.bar(means.index.astype(str), means.values)
                plt.xlabel(target_column)
                plt.ylabel(f"mean({col})")
            plt.title(col)
            plt.grid(True, linestyle="--", alpha=0.5)
        plt.tight_layout()
        plt.show()

## 5. Correlation and leakage checks

We take a quick look at:

- Numeric feature correlation matrix
- Correlation between each feature and the target (for regression-ish targets)

This can reveal:
- Highly correlated features (candidates for removal or regularization)
- Obvious leakage (e.g., feature almost perfectly correlated with target).

In [ ]:
if numeric_cols:
    corr = df_eda[numeric_cols].corr()

    plt.figure(figsize=(6, 5))
    im = plt.imshow(corr, cmap="coolwarm", interpolation="nearest")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=90)
    plt.yticks(range(len(numeric_cols)), numeric_cols)
    plt.title("Correlation matrix (numeric features)")
    plt.tight_layout()
    plt.show()
else:
    print("No numeric columns found for correlation matrix.")

In [ ]:
if target_column is not None and target_column in numeric_cols:
    # Compute absolute correlation of each feature with the numeric target
    target_corr = (
        df_eda[numeric_cols]
        .corr()[target_column]
        .drop(target_column)
        .abs()
        .sort_values(ascending=False)
    )
    display(target_corr.to_frame("|corr_with_target|").head(20))
else:
    print("Target is not numeric (or missing); skipping numeric correlation with target.")

> **Actionable outcomes:**
> - Identify feature pairs with very high correlation (e.g. > 0.95); consider dropping one, or using a model robust to multicollinearity.
> - If a feature is almost perfectly correlated with the target, investigate whether it is **data leakage** (e.g. post-outcome information).

## 6. Prototype feature engineering

This section is for **trying out ideas** like:

- Date decomposition (year, month, day, dayofweek, etc.)
- Simple numeric transforms (log of skewed variables, interactions)
- Simple categorical encodings for EDA (frequency encoding, etc.)

The final *production* feature logic should live in your Python modules (e.g. `ml_tabular.features.build_features`) and be configured via YAML, but the notebook is a good place to experiment first.

In [ ]:
# Example: Identify datetime columns we parsed earlier
datetime_cols = df_eda.select_dtypes(include=["datetime64[ns]"]).columns.tolist()
datetime_cols

In [ ]:
def add_date_parts(df: pd.DataFrame, date_cols: list[str]) -> pd.DataFrame:
    """Add basic date-derived features for given datetime columns.

    This is for exploration; in the template, you could move equivalent
    logic into `ml_tabular.features.build_features`.
    """
    df_copy = df.copy()
    for col in date_cols:
        if col not in df_copy.columns:
            continue
        if not np.issubdtype(df_copy[col].dtype, np.datetime64):
            LOGGER.warning("Column '%s' is not datetime; skipping date parts.", col)
            continue
        base = col
        df_copy[f"{base}_year"] = df_copy[col].dt.year
        df_copy[f"{base}_month"] = df_copy[col].dt.month
        df_copy[f"{base}_day"] = df_copy[col].dt.day
        df_copy[f"{base}_dow"] = df_copy[col].dt.dayofweek
        df_copy[f"{base}_hour"] = df_copy[col].dt.hour
    return df_copy

df_fe = add_date_parts(df_eda, datetime_cols)
df_fe.filter(regex="_year$|_month$|_dow$|_hour$").head()

In [ ]:
# Example: Identify skewed numeric features and log-transform them (for EDA)
numeric_cols_no_target = [c for c in numeric_cols if c != target_column]
skew = df_fe[numeric_cols_no_target].skew().sort_values(ascending=False)
skew.head(10)

In [ ]:
skew_threshold = 1.0
skewed_features = skew[skew.abs() > skew_threshold].index.tolist()
print("Skewed numeric features (|skew| >", skew_threshold, "):", skewed_features)

df_fe_log = df_fe.copy()
for col in skewed_features:
    # Avoid issues with non-positive values; simple example for EDA
    col_min = df_fe_log[col].min()
    shifted = df_fe_log[col] - col_min + 1e-6
    df_fe_log[col + "_log1p"] = np.log1p(shifted)

df_fe_log.filter(regex="_log1p$").head()

You can continue experimenting here:

- Numeric interactions (e.g. `feature1 * feature2`, ratios)
- Frequency encoding for categoricals
- Binning continuous variables

When you find transformations that make sense and are *defensible*, you can move them into your shared feature module so they are **reproducible** for training and inference.

## 7. Reflect decisions into configuration

After EDA, you will often decide:

- Which columns to include / exclude
- Which columns should be treated as numeric / categorical / datetime
- Which engineered features to keep (e.g. date parts, log-transformed features)

You have two main options for recording these decisions:

1. **Update YAML config**: e.g. add lists like `numeric_features`, `categorical_features`, `datetime_features`, `derived_features` under `tabular`.
2. **Update Python feature module**: encode the logic in `ml_tabular.features.build_features` and have the config tell it *what* to do.

Below is a small helper to *export* a candidate feature configuration based on what we just inferred/created. You can then manually review and merge it into your main YAML config.

In [ ]:
import yaml

# Example: propose feature lists

candidate_numeric = [c for c in numeric_cols_no_target if c in df_fe_log.columns]
candidate_categorical = [
    c for c in df_fe_log.select_dtypes(include=["object", "string", "category"]).columns
    if c != target_column
]
candidate_datetime = datetime_cols
candidate_derived = [c for c in df_fe_log.columns if c.endswith("_log1p") or c.endswith("_year") or c.endswith("_month") or c.endswith("_dow") or c.endswith("_hour")]

feature_config = {
    "numeric_features": candidate_numeric,
    "categorical_features": candidate_categorical,
    "datetime_features": candidate_datetime,
    "derived_features": candidate_derived,
}

feature_config

In [ ]:
output_config_path = PROJECT_ROOT / "configs" / "tabular" / "tabular_features_candidate.yaml"

output_config_path.parent.mkdir(parents=True, exist_ok=True)
with output_config_path.open("w", encoding="utf-8") as f:
    yaml.safe_dump(feature_config, f, sort_keys=False)

LOGGER.info("Wrote candidate feature config to: %s", output_config_path)
output_config_path

## 8. Summary

In this notebook, you:

- Loaded config and data through the **same mechanisms** your training code uses
- Inspected schema, types, missing values, and target distribution
- Explored basic feature–target relationships and correlations
- Prototyped reasonable feature engineering ideas (date parts, log transforms)
- Exported a **candidate feature configuration** that you can merge into your main YAML

This demonstrates that you treat **EDA as part of an engineering process**, not just ad-hoc experimentation: insights here feed directly into your config, pipelines, and final models.